In [1]:
import numpy as np
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
import sys

In [2]:
# Set up the driver
chromedriver = "/home/kmussar/Documents/Metis/sea19_ds7/My notes/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)

In [3]:
# df will be used below to collect scraped information
cols = ['trail_name','report_date','trail_condition','url_id','url']
master_df = pd.DataFrame(columns=cols)
master_df

,trail_name,report_date,trail_condition,url_id,url


In [4]:
# Trails to scrape
url_prefix = 'https://www.wta.org/go-hiking/hikes/'
url_end = '/@@related_tripreport_listing?b_start:int='
# lists of urls used in testing/selective scraping
trails_of_interest = ['chilean-memorial','rialto-beach-hole-in-the-wall','quillayute-river','second-beach','toleak-point','third-beach','oil-city']
rattlesnake = ['rattle-snake-ledge']
near_rattlesnake = ['palouse-to-cascades-trail-rattlesnake-lake','twin-falls-state-park','kamikaze-falls','mailbox-peak-old-trail','mount-si-old-trail','olallie-lake-via-pratt-lake-trail','little-si','bandera-mountain']

In [5]:
# update variable after "in" based on the trails you want to scrape
for trail in near_rattlesnake: 
    trail_df = pd.DataFrame(columns=cols)
    trail_dict = {}
    url = url_prefix + trail
    driver.get(url)
    page = driver.page_source
    soup = BeautifulSoup(page, 'lxml')    
    
    # Trail name
    title = soup.find("title")
    title_contents= title.contents
    trail_name = str(title_contents[0]).split(' — ')[0]
    trail_dict['trail_name']= trail_name
    
    # Number of trail reports
    find_num_reports = soup.find("span", class_="ReportCount")
    num_reports= int(find_num_reports.contents[0])
    trail_dict['num_reports']= num_reports
    
    # Trail Rating
    find_rating = soup.find("div",class_="current-rating")
    find_rating_contents = find_rating.contents
    if find_rating_contents == None:
                rating = -1
    else:
        rating = str(find_rating_contents[0])
    trail_dict['rating']= rating
    
    for i in range(int(num_reports/5+1)):
        # point driver to page
        url = url_prefix + trail + url_end + str(i*5) 
        driver.get(url)
        page = driver.page_source
        soup = BeautifulSoup(page, 'lxml')
        reports = soup.find_all("div", class_="item")
        df_page = pd.DataFrame(columns=cols)
        for report in reports:
            df_report = {}
            report_soup = report

            # trip report dates
            hike_date = report_soup.find("p", class_=re.compile("hike-date"))

            date = str(list(hike_date)[1]).split('"')[3].strip()
            df_report['report_date'] = date

            # trail conditions
            trail_issues = report_soup.find("div", class_="trail-issues")
            if trail_issues == None:
                df_report['trail_condition'] = ''
            else:
                condition = trail_issues.text.split(':')[1].strip()
                df_report['trail_condition'] = condition

            # Report URLS and URL IDs
            report_url = report_soup.find("a", class_=re.compile("show-with-full"))
            if report_url == None: 
                df_report['url'] = ''
                df_report['url_id'] = ''
            else:
                df_report['url'] = report_url['href']
                url_id = report_url['href'][-21:]
                df_report['url_id'] = url_id
            df_page = df_page.append(df_report, ignore_index=True)
            time.sleep(.5)
        df_page['trail_name'] = trail_name
        df_page['num_reports']= num_reports
        df_page['rating']= rating
        trail_df = trail_df.append(df_page, ignore_index=True)
            
    time.sleep(1)
    #print(trail_dict)
    master_df = master_df.append(trail_df, ignore_index=True)
    #master_df = master_df.append(trail_dict, ignore_index=True)

master_df.head()  
    

/home/kmussar/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,num_reports,rating,report_date,trail_condition,trail_name,url,url_id
0,119.0,2.67 out of 5,"Oct 06, 2018",,Palouse to Cascades Trail - Rattlesnake Lake,https://www.wta.org/go-hiking/trip-reports/tri...,2018-10-07.9502327131
1,119.0,2.67 out of 5,"Aug 26, 2018",,Palouse to Cascades Trail - Rattlesnake Lake,https://www.wta.org/go-hiking/trip-reports/tri...,2018-08-26.6172090227
2,119.0,2.67 out of 5,"Jul 17, 2018",,Palouse to Cascades Trail - Rattlesnake Lake,https://www.wta.org/go-hiking/trip-reports/tri...,2018-07-17.2167806412
3,119.0,2.67 out of 5,"Jun 21, 2018",bugs,Palouse to Cascades Trail - Rattlesnake Lake,https://www.wta.org/go-hiking/trip-reports/tri...,2018-06-22.2974435949
4,119.0,2.67 out of 5,"Apr 22, 2018",,Palouse to Cascades Trail - Rattlesnake Lake,https://www.wta.org/go-hiking/trip-reports/tri...,2018-04-22.9805279293


In [7]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4954 entries, 0 to 4953
Data columns (total 7 columns):
num_reports        4954 non-null float64
rating             4954 non-null object
report_date        4954 non-null object
trail_condition    4954 non-null object
trail_name         4954 non-null object
url                4954 non-null object
url_id             4954 non-null object
dtypes: float64(1), object(6)
memory usage: 271.0+ KB


In [6]:
# pickle results from scraping!
import pickle

with open('near_rattlesnake.pickle', 'wb') as to_write:
    pickle.dump(master_df, to_write)